**Exercise 3**

Let $ x_1, x_2, x_3, x_4 and x_5 $ be the number of drone of type MANAS, JAY, VEER, SPASHT AND DHAKSH respectively.

objective function : max $12x_1+14x_2+10x_3+15x_4+14x_5 \\ $
subjected to : $ 6x_1 +7.5x_2 + 9.5x_3+ 10x_4 + 12.5x_5 \leq 59 \\ 4.25x_1 + 3.25x_2 + 2.5x_3 + 3.45x_4 + 2.25x_5 \leq 73 $

and $ x_i\text's $ is from whole number 


In [1]:
!pip install -q pyomo
from pyomo.environ import *
import numpy as np 
import pandas as pd

     |████████████████████████████████| 9.1 MB 5.2 MB/s 
     |████████████████████████████████| 49 kB 4.9 MB/s 


In [2]:
data_csvfile = pd.read_csv("lab4 ex3.csv")

In [3]:
model = ConcreteModel()

In [4]:
N = len(data_csvfile.index)

In [5]:
M = 2

In [6]:
col_indices = range(N)

In [7]:
model.x = Var(col_indices, domain = NonNegativeIntegers)

In [8]:
model.objective = Objective(expr = summation(data_csvfile['Carrying Capacity'], model.x), sense = maximize)

In [9]:
model.constraints = ConstraintList()

In [10]:
model.constraints.add(summation(data_csvfile['Cost'],model.x) <= 59)
model.constraints.add(summation(data_csvfile['Parking space'],model.x) <= 73)

In [11]:
upper_bound = np.inf
for i in col_indices:
  model.x[i].setub(upper_bound)

In [12]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True : NonNegativeIntegers
          1 :     0 :  None :   inf : False :  True : NonNegativeIntegers
          2 :     0 :  None :   inf : False :  True : NonNegativeIntegers
          3 :     0 :  None :   inf : False :  True : NonNegativeIntegers
          4 :     0 :  None :   inf : False :  True : NonNegativeIntegers

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 15*x[3] + 

In [13]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 148489 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [14]:
opt_cbc = SolverFactory('cbc')

In [15]:
result1 = opt_cbc.solve(model)

In [16]:
print("6. Objective function Value: ", model.objective())
print(" Value of decision variable: ")
for i in col_indices:
  print("Number of ", data_csvfile['Drone Model'][i],"drone", model.x[i].value)

 Objective function Value:  114.0
 Value of decision variable: 
Number of  MANAS drone 6.0
Number of  JAY drone 3.0
Number of  VEER drone 0.0
Number of  SPASHT drone 0.0
Number of  DHAKSH drone 0.0


Now removing restriction that variables are from integers and solving the linear program 

In [17]:
for i in col_indices:
  model.x[i].domain = Reals
  model.x[i].setlb(0)

In [18]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   6.0 :   inf : False : False :  Reals
          1 :     0 :   3.0 :   inf : False : False :  Reals
          2 :     0 :   0.0 :   inf : False : False :  Reals
          3 :     0 :   0.0 :   inf : False : False :  Reals
          4 :     0 :   0.0 :   inf : False : False :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 14*x[1] + 10*x[2] + 15*x[3] + 14*x[4]

1 Constraint Declarations
    constraints : Size=2, Inde

In [19]:
result2 = opt_cbc.solve(model)

In [47]:
print("7. Objective function value :",model.objective())
print(" Value of decision variable: ")
for i in col_indices:
  print("Number of ", data_csvfile['Drone Model'][i],"drone", model.x[i].value)

7. Objective function value : 122.000004
 Value of decision variable: 
Number of  MANAS drone 10.166667
Number of  JAY drone 0.0
Number of  VEER drone 0.0
Number of  SPASHT drone 0.0
Number of  DHAKSH drone 0.0


no solution of MILP can't be obtained by rounding off the solution of LP because there is significant difference in both the values 

after rounding off soltuion of LP we get 122 but soltuion of MILP is 114  

9. when thw budget is increased to 71 crores and parking space in increased to 87 units 
new constraint are :
$ 6x_1 +7.5x_2 + 9.5x_3+ 10x_4 + 12.5x_5 \leq 71 \\ 4.25x_1 + 3.25x_2 + 2.5x_3 + 3.45x_4 + 2.25x_5 \leq 87 $

In [22]:
for i in range(2):
  model.constraints[i+1].deactivate()

In [48]:
for i in col_indices:
  model.x[i].domain = NonNegativeIntegers

In [25]:
model.constraints.add(summation(data_csvfile['Cost'],model.x) <= 71)
model.constraints.add(summation(data_csvfile['Parking space'],model.x) <= 87)

In [26]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value     : Upper : Fixed : Stale : Domain
          0 :     0 : 9.8333333 :   inf : False : False : NonNegativeIntegers
          1 :     0 :       0.0 :   inf : False : False : NonNegativeIntegers
          2 :     0 :       0.0 :   inf : False : False : NonNegativeIntegers
          3 :     0 :       0.0 :   inf : False : False : NonNegativeIntegers
          4 :     0 :       0.0 :   inf : False : False : NonNegativeIntegers

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 12*x[0] + 

In [27]:
result3 = opt_cbc.solve(model)

In [28]:
print("9. Objective function value: ", model.objective())
print(" Value of decision variable: ")
for i in col_indices:
  print("Number of ", data_csvfile['Drone Model'][i],"drone", model.x[i].value)

Objective function value:  138.0
 Value of decision variable: 
Number of  MANAS drone 8.0
Number of  JAY drone 3.0
Number of  VEER drone 0.0
Number of  SPASHT drone 0.0
Number of  DHAKSH drone 0.0


Now removing restriction from variables and solving linear problem 

In [29]:
for i in col_indices:
  model.x[i].domain = Reals
  model.x[i].setlb(0)

In [30]:
result4 = opt_cbc.solve(model)

In [31]:
print("9. Objective function value: ", model.objective())
print(" Value of decision variable: ")
for i in col_indices:
  print("Number of ", data_csvfile['Drone Model'][i],"drone", model.x[i].value)

Objective function value:  141.999996
 Value of decision variable: 
Number of  MANAS drone 11.833333
Number of  JAY drone 0.0
Number of  VEER drone 0.0
Number of  SPASHT drone 0.0
Number of  DHAKSH drone 0.0


As the Budget and parking space increases the carrying capacity also increased 

10. Now changing rhs of constraints :

Budget : 67 crores

parking space: 88 units

$ 6x_1 +7.5x_2 + 9.5x_3+ 10x_4 + 12.5x_5 \leq 67 \\ 4.25x_1 + 3.25x_2 + 2.5x_3 + 3.45x_4 + 2.25x_5 \leq 88 $

In [38]:
for i in col_indices:
  model.x[i].domain = NonNegativeIntegers

for i in range(2,4):
  model.constraints[i+1].deactivate()

model.constraints.add(summation(data_csvfile['Cost'],model.x) <= 67)
model.constraints.add(summation(data_csvfile['Parking space'],model.x) <= 88)

result5 = opt_cbc.solve(model)

print("Objective function value: ", model.objective())
print(" Value of decision variable: ")
for i in col_indices:
  print("Number of ", data_csvfile['Drone Model'][i],"drone", model.x[i].value)

Objective function value:  132.0
 Value of decision variable: 
Number of  MANAS drone 11.0
Number of  JAY drone 0.0
Number of  VEER drone 0.0
Number of  SPASHT drone 0.0
Number of  DHAKSH drone 0.0


Now removing restriction that variables are from integers and solving the linear program 

In [39]:
for i in col_indices:
  model.x[i].domain = Reals
  model.x[i].setlb(0)
result6 = opt_cbc.solve(model)

print("Objective function value: ", model.objective())
print(" Value of decision variable: ")
for i in col_indices:
  print("Number of ", data_csvfile['Drone Model'][i],"drone", model.x[i].value)

Objective function value:  134.000004
 Value of decision variable: 
Number of  MANAS drone 11.166667
Number of  JAY drone 0.0
Number of  VEER drone 0.0
Number of  SPASHT drone 0.0
Number of  DHAKSH drone 0.0


Budget : 65 crores

parking space: 89 units

$ 6x_1 +7.5x_2 + 9.5x_3+ 10x_4 + 12.5x_5 \leq 65 \\ 4.25x_1 + 3.25x_2 + 2.5x_3 + 3.45x_4 + 2.25x_5 \leq 89 $

In [41]:
for i in col_indices:
  model.x[i].domain = NonNegativeIntegers

for i in range(4,6):
  model.constraints[i+1].deactivate()

model.constraints.add(summation(data_csvfile['Cost'],model.x) <= 65)
model.constraints.add(summation(data_csvfile['Parking space'],model.x) <= 89)

result7 = opt_cbc.solve(model)

print("Objective function value: ", model.objective())
print(" Value of decision variable: ")
for i in col_indices:
  print("Number of ", data_csvfile['Drone Model'][i],"drone", model.x[i].value)

Objective function value:  126.0
 Value of decision variable: 
Number of  MANAS drone 7.0
Number of  JAY drone 3.0
Number of  VEER drone 0.0
Number of  SPASHT drone 0.0
Number of  DHAKSH drone 0.0


Now removing restriction that variables are from integers and solving the linear program 

In [42]:
for i in col_indices:
  model.x[i].domain = Reals
  model.x[i].setlb(0)
result8 = opt_cbc.solve(model)

print("Objective function value: ", model.objective())
print(" Value of decision variable: ")
for i in col_indices:
  print("Number of ", data_csvfile['Drone Model'][i],"drone", model.x[i].value)

Objective function value:  129.999996
 Value of decision variable: 
Number of  MANAS drone 10.833333
Number of  JAY drone 0.0
Number of  VEER drone 0.0
Number of  SPASHT drone 0.0
Number of  DHAKSH drone 0.0


Budget : 63 crores

parking space: 90 units

$ 6x_1 +7.5x_2 + 9.5x_3+ 10x_4 + 12.5x_5 \leq 63 \\ 4.25x_1 + 3.25x_2 + 2.5x_3 + 3.45x_4 + 2.25x_5 \leq 90 $

In [43]:
for i in col_indices:
  model.x[i].domain = NonNegativeIntegers

for i in range(6,8):
  model.constraints[i+1].deactivate()

model.constraints.add(summation(data_csvfile['Cost'],model.x) <= 63)
model.constraints.add(summation(data_csvfile['Parking space'],model.x) <= 90)

result9 = opt_cbc.solve(model)

print("Objective function value: ", model.objective())
print(" Value of decision variable: ")
for i in col_indices:
  print("Number of ", data_csvfile['Drone Model'][i],"drone", model.x[i].value)

Objective function value:  124.0
 Value of decision variable: 
Number of  MANAS drone 8.0
Number of  JAY drone 2.0
Number of  VEER drone 0.0
Number of  SPASHT drone 0.0
Number of  DHAKSH drone 0.0


Now removing restriction that variables are from integers and solving the linear program 

In [44]:
for i in col_indices:
  model.x[i].domain = Reals
  model.x[i].setlb(0)
result10 = opt_cbc.solve(model)

print("Objective function value: ", model.objective())
print(" Value of decision variable: ")
for i in col_indices:
  print("Number of ", data_csvfile['Drone Model'][i],"drone", model.x[i].value)

Objective function value:  126.0
 Value of decision variable: 
Number of  MANAS drone 10.5
Number of  JAY drone 0.0
Number of  VEER drone 0.0
Number of  SPASHT drone 0.0
Number of  DHAKSH drone 0.0


Budget : 61 crores

parking space: 91 units

$ 6x_1 +7.5x_2 + 9.5x_3+ 10x_4 + 12.5x_5 \leq 61 \\ 4.25x_1 + 3.25x_2 + 2.5x_3 + 3.45x_4 + 2.25x_5 \leq 91 $

In [49]:
for i in col_indices:
  model.x[i].domain = NonNegativeIntegers

for i in range(8,10):
  model.constraints[i+1].deactivate()

model.constraints.add(summation(data_csvfile['Cost'],model.x) <= 61)
model.constraints.add(summation(data_csvfile['Parking space'],model.x) <= 91)

result11 = opt_cbc.solve(model)

print("Objective function value: ", model.objective())
print(" Value of decision variable: ")
for i in col_indices:
  print("Number of ", data_csvfile['Drone Model'][i],"drone", model.x[i].value)

Objective function value:  120.0
 Value of decision variable: 
Number of  MANAS drone 10.0
Number of  JAY drone 0.0
Number of  VEER drone 0.0
Number of  SPASHT drone 0.0
Number of  DHAKSH drone 0.0


Now removing restriction that variables are from integers and solving the linear program 

In [50]:
for i in col_indices:
  model.x[i].domain = Reals
  model.x[i].setlb(0)
result12 = opt_cbc.solve(model)

print("Objective function value: ", model.objective())
print(" Value of decision variable: ")
for i in col_indices:
  print("Number of ", data_csvfile['Drone Model'][i],"drone", model.x[i].value)

Objective function value:  122.000004
 Value of decision variable: 
Number of  MANAS drone 10.166667
Number of  JAY drone 0.0
Number of  VEER drone 0.0
Number of  SPASHT drone 0.0
Number of  DHAKSH drone 0.0


Solution of MILP and LP are nearly close and it is seen that the value of objective function in LP is always greater than objective in MILP 

Value of objective function is mainly dependent on $x_0$ in both MILP and LP 